# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3148395b80>
├── 'a' --> tensor([[ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363]])
└── 'x' --> <FastTreeValue 0x7f3148395a60>
    └── 'c' --> tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                        [-0.1038,  1.3954, -0.9500,  0.1306],
                        [ 0.4227,  2.7902, -1.1884,  0.4287]])

In [4]:
t.a

tensor([[ 1.0550,  0.4666,  0.5489],
        [-1.3435, -0.1555, -1.9363]])

In [5]:
%timeit t.a

64.3 ns ± 0.0742 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3148395b80>
├── 'a' --> tensor([[ 0.3808, -0.4842, -1.1928],
│                   [ 0.1311, -0.0126,  0.8680]])
└── 'x' --> <FastTreeValue 0x7f3148395a60>
    └── 'c' --> tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                        [-0.1038,  1.3954, -0.9500,  0.1306],
                        [ 0.4227,  2.7902, -1.1884,  0.4287]])

In [7]:
%timeit t.a = new_value

68.3 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0550,  0.4666,  0.5489],
               [-1.3435, -0.1555, -1.9363]]),
    x: Batch(
           c: tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0550,  0.4666,  0.5489],
        [-1.3435, -0.1555, -1.9363]])

In [11]:
%timeit b.a

58.2 ns ± 0.0687 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2982, -0.8270, -1.4593],
               [-0.3584,  0.1151,  0.6594]]),
    x: Batch(
           c: tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 0.578 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 671 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f30721815e0>
├── 'a' --> tensor([[[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]],
│           
│                   [[ 1.0550,  0.4666,  0.5489],
│                    [-1.3435, -0.1555, -1.9363]]])
└── 'x' --> <FastTreeValue 0x7f314c441d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 50.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f31483ac580>
├── 'a' --> tensor([[ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363],
│                   [ 1.0550,  0.4666,  0.5489],
│                   [-1.3435, -0.1555, -1.9363]])
└── 'x' --> <FastTreeValue 0x7f3072181370>
    └── 'c' --> tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                        [-0.1038,  1.3954, -0.9500,  0.1306],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 21.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.9 µs ± 56.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1466,  0.1633, -0.5746,  0.0572],
                       [-0.1038,  1.3954, -0.9500,  0.1306],
                       [ 0.4227,  2.7902, -1.1884,  0.4287]],
              
                      [[-0.1466,  0.1633, -0.5746,  0.0572],
                       [-0.1038,  1.3954, -0.9500,  0.1306],
                       [ 0.4227,  2.7902, -1.1884,  0.4287]],
              
                      [[-0.1466,  0.1633, -0.5746,  0.0572],
                       [-0.1038,  1.3954, -0.9500,  0.1306],
                       [ 0.4227,  2.7902, -1.1884,  0.4287]],
              
                      [[-0.1466,  0.1633, -0.5746,  0.0572],
                       [-0.1038,  1.3954, -0.9500,  0.1306],
                       [ 0.4227,  2.7902, -1.1884,  0.4287]],
              
                      [[-0.1466,  0.1633, -0.5746,  0.0572],
                       [-0.1038,  1.3954, -0.9500,  0.1306],
                       [ 0.4227,  2.7902, -1.1884,  0.4287]],

In [26]:
%timeit Batch.stack(batches)

79.5 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287],
                      [-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287],
                      [-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287],
                      [-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287],
                      [-0.1466,  0.1633, -0.5746,  0.0572],
                      [-0.1038,  1.3954, -0.9500,  0.1306],
                      [ 0.4227,  2.7902, -1.1884,  0.4287],
                      [-0.1466,  0.1633, -0.5746,  0.0572],
                   

In [28]:
%timeit Batch.cat(batches)

150 µs ± 896 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

350 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
